In [8]:
from pyspark.sql.functions import udf, size, avg, count, col,sum, explode
import json
from datetime import datetime


# env = "production"
env = "staging"

s3_bucket = "s3://mist-aggregated-stats-{env}/aggregated-stats/".format(env=env)
date_day = "2020-05-*"
hr = "*" #now().hour-1

rrm_radar_path = s3_bucket + "rrm_radar_channel_stat/dt={day}/hr={hr}/*.csv".format(env=env, day=date_day, hr=hr)
print(rrm_radar_path)


date_day = datetime.today().strftime('%Y-%m-%d') #"2020-05-*"
curr_hr = '*'  # latest 
ap_neighbors_path = s3_bucket + "top_1_time_epoch_by_site_ap_ap2_band/dt={day}/hr={hr}/*.csv".format(env=env, day=date_day, hr=hr)

print(ap_neighbors_path)


s3://mist-aggregated-stats-staging/aggregated-stats/rrm_radar_channel_stat/dt=2020-05-*/hr=*/*.csv
s3://mist-aggregated-stats-staging/aggregated-stats/top_1_time_epoch_by_site_ap_ap2_band/dt=2020-05-21/hr=*/*.csv

In [11]:
# # rdd_rrm_local = spark.sparkContext.sequenceFile(rrm_local_path)
# rdd_rrm_events = spark.sparkContext.textFile(rrm_radar_path)
# rdd_rrm_events.take(10)

df_radar = spark.read.format("csv")\
   .option("header", "true")\
   .option("inferSchema", "true")\
   .load(rrm_radar_path)
df_radar.createOrReplaceTempView("scan_data")
df_Schema = df_radar.schema
df_radar.show(5)

+--------------------+------------+-----------+-------------------+---------------+----------+
|             site_id|       ap_id|pre_channel|radar_channel_count|last_radar_time|      time|
+--------------------+------------+-----------+-------------------+---------------+----------+
|c943d077-1b67-491...|5c5b35500622|        104|                  1|     1589096727|1589094000|
|c943d077-1b67-491...|5c5b35500924|        116|                  1|     1589095299|1589094000|
|c943d077-1b67-491...|5c5b35500686|        104|                  1|     1589095247|1589094000|
|c943d077-1b67-491...|5c5b35500587|         64|                  1|     1589096287|1589094000|
|c943d077-1b67-491...|5c5b35500839|         64|                  1|     1588919882|1588917600|
+--------------------+------------+-----------+-------------------+---------------+----------+
only showing top 5 rows

# Impacted Sites

In [12]:
#impacted Sites
df_radar_sites = df_radar.select('site_id', 'radar_channel_count').groupBy("site_id").count()
df_radar_sites.show()

+--------------------+-----+
|             site_id|count|
+--------------------+-----+
|ee915a44-b963-11e...|   11|
|f688779c-e335-4f8...|    3|
|70e0f468-fc13-11e...|    1|
|e80749be-963c-4d2...|    2|
|6db15e18-bc94-11e...|    2|
|67970e46-4e12-11e...|   15|
|fc656275-b157-43f...|    1|
|9c3f2be6-397d-11e...|    2|
|35670395-458f-4ec...|    2|
|d23f418c-a5ab-4e3...|    6|
|a8178443-ecb5-461...|   21|
|20325077-2d75-4fa...|    3|
|96ca2c79-b7db-402...|    1|
|d7c802c4-1d3a-4aa...|   10|
|12c07c11-51a3-495...|    3|
|f733a5d7-5179-4df...|    2|
|fe1f336e-fba7-44d...|    1|
|d59ee965-e757-45a...|    1|
|b0bbfdc0-aaba-11e...|    1|
|4a6427ac-4514-4ab...|    1|
+--------------------+-----+
only showing top 20 rows

In [14]:
site_counter_filter = 10   # change based on #APs_in_site 

impacted_sites = df_radar_sites.filter(col('count')>site_counter_filter).select("site_id", "count").collect()
impacted_sites

[Row(site_id='ee915a44-b963-11e5-b14c-1258369c38a9', count=11), Row(site_id='67970e46-4e12-11e6-9188-0242ac110007', count=15), Row(site_id='a8178443-ecb5-461c-b854-f16627619ab3', count=21), Row(site_id='c943d077-1b67-4915-a690-e698543d2790', count=115)]

# Check one Impacted-site

In [15]:
impacted_site_id = impacted_sites[0]['site_id']

In [16]:
#impacted APs
df_radar_in_site = df_radar.filter(col("site_id")==impacted_site_id).select('ap_id',col("pre_channel").alias("radar_channel"), 'last_radar_time')

df_radar_in_site.show()
# df_radar_ap_lastseen.show() )

# df_radar_ap_lastseen = df_radar_in_site.select('pre_channel', 'last_radar_time').groupBy("ap_id", col("pre_channel").alias("radar_channel"), 'last_radar_time').max()
# df_radar_ap_lastseen.show()

# df_radar_ap_lastseen = df_radar.filter(col("site_id")==impacted_site_id).select("ap_id", 'pre_channel', 'last_radar_time').groupBy("ap_id", col("pre_channel").alias("radar_channel"), 'last_radar_time').max()
# df_radar_ap_lastseen.show()

+------------+-------------+---------------+
|       ap_id|radar_channel|last_radar_time|
+------------+-------------+---------------+
|5c5b3550032f|          100|     1588928836|
|5c5b3550032f|          100|     1588938523|
|5c5b3550032f|          100|     1588934289|
|5c5b3550032f|          132|     1588948689|
|5c5b3550032f|          132|     1589037054|
|5c5b3550032f|           52|     1589209332|
|5c5b3550032f|           52|     1588952779|
|5c5b3550032f|          132|     1588969154|
|5c5b3550032f|           52|     1588960942|
|5c5b3550032f|          116|     1588963397|
|5c5b3550032f|          100|     1589019556|
+------------+-------------+---------------+

In [24]:
# add hour_of_day, day_of_week
from pyspark.sql.types import IntegerType

get_hour = udf(lambda t: datetime.fromtimestamp(t).hour, IntegerType())
get_weekday = udf(lambda t: datetime.fromtimestamp(t).weekday(), IntegerType())
df_radar_in_site= df_radar_in_site.withColumn("hour_of_day", get_hour(col("last_radar_time")))\
                                  .withColumn("day_of_week", get_weekday(col("last_radar_time")))


df_radar_in_site_count = df_radar_in_site.select("ap_id", 'radar_channel', "day_of_week", "hour_of_day")\
                                        .groupBy("ap_id", 'radar_channel', "day_of_week", "hour_of_day")\
                                        .count()
df_radar_in_site_count.show()

+------------+-------------+-----------+-----------+-----+
|       ap_id|radar_channel|day_of_week|hour_of_day|count|
+------------+-------------+-----------+-----------+-----+
|5c5b3550032f|          132|          4|         14|    1|
|5c5b3550032f|          100|          5|         10|    1|
|5c5b3550032f|          100|          4|         11|    1|
|5c5b3550032f|          132|          4|         20|    1|
|5c5b3550032f|          132|          5|         15|    1|
|5c5b3550032f|           52|          4|         18|    1|
|5c5b3550032f|          100|          4|         10|    1|
|5c5b3550032f|          116|          4|         18|    1|
|5c5b3550032f|           52|          4|         15|    1|
|5c5b3550032f|          100|          4|          9|    1|
|5c5b3550032f|           52|          0|         15|    1|
+------------+-------------+-----------+-----------+-----+

## Impacted APs, channel

In [26]:
df_radar_by_ap = df_radar_in_site.select('ap_id', 'radar_channel')\
                                .groupBy('ap_id', 'radar_channel')\
                                .count()
df_radar_by_ap.show()


+------------+-------------+-----+
|       ap_id|radar_channel|count|
+------------+-------------+-----+
|5c5b3550032f|          116|    1|
|5c5b3550032f|           52|    3|
|5c5b3550032f|          132|    3|
|5c5b3550032f|          100|    4|
+------------+-------------+-----+

In [27]:
df_radar_by_channel = df_radar_in_site.select('radar_channel', 'last_radar_time')\
                                    .groupBy('radar_channel')\
                                    .agg({'last_radar_time':'max'})
df_radar_by_channel.show()

+-------------+--------------------+
|radar_channel|max(last_radar_time)|
+-------------+--------------------+
|          132|          1589037054|
|           52|          1589209332|
|          100|          1589019556|
|          116|          1588963397|
+-------------+--------------------+

In [29]:
# from pyspark.sql.types import *
# time_to_hour = udf(lambda t: int(t)//3600*3600, IntegerType())

df_radar_ch_by_hour= df_radar_in_site.select('hour_of_day').groupBy('hour_of_day').count()
df_radar_ch_by_hour.show()

+-----------+-----+
|hour_of_day|count|
+-----------+-----+
|         20|    1|
|         15|    3|
|          9|    1|
|         10|    2|
|         11|    1|
|         14|    1|
|         18|    2|
+-----------+-----+

In [14]:
# df_radar_ch_by_hour.groupBy("last_radar_time").count().show()

# from pyspark.sql.functions  import hour,minute,second
# # import org.apache.spark.sql.functions.{col,}
# # from pyspark.sql.functions import date_format
# # df_radar_in_site.withColumn("hour", hour(col("last_radar_time").cast(TimestampType))).show()
# df_radar_in_site.select(col("last_radar_time").cast(TimestampType))


# de-Weight function

In [30]:
"""


Radar_detected AP1

Wt_t1(ap1) = w_t0(ap2)* weight_decay(t1, site_last_seen) + weight(channel, map)
UNII-22 channel, weather_radio ,  



Neighbors: 
Wt_t1 (ap2)  = wt_t(ap2)  + weight_From_neighbor (rssi_ap1_ap2)  

Functions:

weight_decay(t1, last_seen) 
weight_0 * exp(- t/decay_lifetime),   decay_lifetime = 1 week,

weight(channel, map)
DFS Channel, 
Weather channel
 weight_From_neighbor (rssi) 
Weight_from_connectivity (SLE anomaly) 


Goal:  (org, site, ap, channel, weight_0, last_seen) in Redis

"""

import numpy as np
def weight_by_dfs_channel(chn, wt_0=0.4, wt_weather_ch=0.1 ):
    if 116<=chn<132:
        wt = wt_0 + wt_weather_ch
    else:
        wt = wt_0
    return wt

def weight_by_dist(dist_to_airport=100.0, max_wt=0.8):
    wt = 0.0
    if dist_to_airport< 10:
        wt = max_wt * np.exp(-dist_to_airport**2/5**2)
    
    return wt

def weight_impacted_neighbor(rssi, rssi_0 = -65):
    wt = 1.0
    if rssi < rssi_0:
        wt =  np.exp((rssi - rssi_0)/15) 
    return wt
    
    
def weight_tot(ch, rssi=-40, dist_to_airport=100.0, theta=[0.33, 0.33, 0.33]):
    wt=[0, 0, 0]
    wt[0] = weight_by_dfs_channel(ch)
    wt[1] = weight_by_dist(dist_to_airport)
    wt[2] = weight_impacted_neighbor(rssi)
    
    wt_0 = 0.0
    for i in range(2):
        wt_0 += wt[i] * theta[i]
    
#     print(wt_0, wt)
#     wt_0 = 1.0/(1.0 + np.exp(-0.6*wt_0))
    wt_0 = (wt[0] + wt[1]) * wt[2]

    return wt_0


In [31]:
ch = 40
d = 100
for r in range(-85, -50, 5):
    print(r, weight_tot(ch, r))

-85 0.10543885524629071
-80 0.14715177646857694
-75 0.2053668476130368
-70 0.2866125242295157
-65 0.4
-60 0.4
-55 0.4

In [34]:
for dist in range(0, 20):
    print(m, "miles",  weight_by_dist(dist))
    

-75 miles 0.8
-75 miles 0.7686315513218586
-75 miles 0.6817150311729692
-75 miles 0.5581410608568248
-75 miles 0.42183393923443885
-75 miles 0.2943035529371539
-75 miles 0.18954220694569743
-75 miles 0.112686736736836
-75 miles 0.0618437923546398
-75 miles 0.031331116079189654
-75 miles 0.0
-75 miles 0.0
-75 miles 0.0
-75 miles 0.0
-75 miles 0.0
-75 miles 0.0
-75 miles 0.0
-75 miles 0.0
-75 miles 0.0
-75 miles 0.0

In [35]:
for m in range(-40, -80, -5):
    print(m, "db",  weight_impacted_neighbor(m, -50))
    

-40 db 1.0
-45 db 1.0
-50 db 1.0
-55 db 0.7165313105737893
-60 db 0.513417119032592
-65 db 0.36787944117144233
-70 db 0.26359713811572677
-75 db 0.18887560283756183

In [36]:
# df_radar_in_site = df_radar.filter(col("site_id")==impacted_site_id)
# df_radar_in_site.groupBy("ap_id", col("pre_channel").alias("radar_channel")).agg({'radar_channel':'count', 'last_radar_time':'max'})
# df_radar_ap_in_site.show()

In [37]:
"""
Impacting on weight and  (Org, site, ap1, ap2, rssi ) () s3://mist-aggregated-stats-production/aggregated-stats/top_1_time_epoch_by_site_ap_ap2_band/
Radar_detected AP
Wt_t1(ap0) = w_t0(ap0)* weight_decay(t1, last_seen) + weight(channel, map)
Neighbors: 
Wt_t1 (ap1)  = wt_t(ap1)  + weight_From_neighbor (rssi)  
Functions:
weight_decay(t1, last_seen) 
weight_0 * exp(- t/decay_lifetime),   decay_lifetime = 1 week,
weight(channel, map)
DFS Channel, 
Weather channel
 weight_From_neighbor (rssi) 
Weight_from_connectivity (SLE anomaly) 
Goal:  (org, site, ap, channel, weight_0, last_seen) in Redis

"""

'\nImpacting on weight and  (Org, site, ap1, ap2, rssi ) () s3://mist-aggregated-stats-production/aggregated-stats/top_1_time_epoch_by_site_ap_ap2_band/\nRadar_detected AP\nWt_t1(ap0) = w_t0(ap0)* weight_decay(t1, last_seen) + weight(channel, map)\nNeighbors: \nWt_t1 (ap1)  = wt_t(ap1)  + weight_From_neighbor (rssi)  \nFunctions:\nweight_decay(t1, last_seen) \nweight_0 * exp(- t/decay_lifetime),   decay_lifetime = 1 week,\nweight(channel, map)\nDFS Channel, \nWeather channel\n weight_From_neighbor (rssi) \nWeight_from_connectivity (SLE anomaly) \nGoal:  (org, site, ap, channel, weight_0, last_seen) in Redis\n\n'

In [38]:
df_radar_in_site.show()

+------------+-------------+---------------+-----------+-----------+
|       ap_id|radar_channel|last_radar_time|hour_of_day|day_of_week|
+------------+-------------+---------------+-----------+-----------+
|5c5b3550032f|          100|     1588928836|          9|          4|
|5c5b3550032f|          100|     1588938523|         11|          4|
|5c5b3550032f|          100|     1588934289|         10|          4|
|5c5b3550032f|          132|     1588948689|         14|          4|
|5c5b3550032f|          132|     1589037054|         15|          5|
|5c5b3550032f|           52|     1589209332|         15|          0|
|5c5b3550032f|           52|     1588952779|         15|          4|
|5c5b3550032f|          132|     1588969154|         20|          4|
|5c5b3550032f|           52|     1588960942|         18|          4|
|5c5b3550032f|          116|     1588963397|         18|          4|
|5c5b3550032f|          100|     1589019556|         10|          5|
+------------+-------------+------

In [22]:
def weight_by_dfs_channel(chn, wt_0=0.4, wt_weather_ch=0.1 ):
    if 116<=chn<132:
        wt = wt_0 + wt_weather_ch
    else:
        wt = wt_0
    return wt

weight_channel = udf(lambda ch: weight_by_dfs_channel(ch, 0.6, 0.2))
df_radar_in_site_weight = df_radar_in_site.withColumn("weight", weight_channel(col('radar_channel')))
df_radar_in_site_weight= df_radar_in_site_weight.select("ap_id", "radar_channel", "weight").groupBy("ap_id", "radar_channel").agg({"weight":'max'})
df_radar_in_site_weight.show()


+------------+-------------+-----------+
|       ap_id|radar_channel|max(weight)|
+------------+-------------+-----------+
|5c5b3550032f|          116|        0.8|
|5c5b3550032f|           52|        0.6|
|5c5b3550032f|          132|        0.6|
|5c5b3550032f|          100|        0.6|
+------------+-------------+-----------+

# Impacts on neighbors

In [39]:
# rdd_ap_neigbhors = spark.sparkContext.textFile(ap_neighbors_path)
# rdd_ap_neigbhors.take(3)

In [40]:
df_scan = spark.read.format("csv")\
   .option("header", "true")\
   .option("inferSchema", "true")\
   .load(ap_neighbors_path)
df_scan.createOrReplaceTempView("scan_data")
df_Schema = df_scan.schema

In [44]:
df_scan.show(10, False)

+------------------------------------+------------+------------+----+----------+-----+
|site                                |ap          |ap2         |band|time_epoch|rssi |
+------------------------------------+------------+------------+----+----------+-----+
|12c07c11-51a3-4958-9aee-8bf0f7ff193c|5c5b354e318d|5c5b351ebf9a|24  |1590076498|-60.0|
|26c0c429-933a-4fda-b230-50365a36c264|5c5b35521c14|5c5b355219cb|24  |1590076663|-49.0|
|57b2f891-09c1-4dcc-8ff1-2f9fb3ff7d39|5c5b353e4f47|5c5b350ede8c|5   |1590076368|-44.0|
|6db15e18-bc94-11e5-91a9-1258369c38a9|5c5b350e070d|5c5b350e0951|24  |1590076706|-43.0|
|6db15e18-bc94-11e5-91a9-1258369c38a9|5c5b3550036b|5c5b351ebf9a|24  |1590076682|-69.0|
|70e0f468-fc13-11e5-85ad-0242ac110008|5c5b350eb3f2|5c5b350e02b7|24  |1590076649|-60.0|
|76d69c3c-05ae-4f8a-ae16-0c0898c2f2f0|5c5b350e0659|5c5b350e141e|24  |1590076796|-39.0|
|997e7de5-794f-4c0c-b722-8f97c7cf27a4|5c5b353e4f10|5c5b350e4cb8|24  |1590076613|-60.0|
|a8178443-ecb5-461c-b854-f16627619ab3|5c5b3

In [46]:
df_scan_in_site = df_scan.filter(col("site")==impacted_site_id)
df_scan_in_site.show(10, False)

+------------------------------------+------------+------------+----+----------+-----+
|site                                |ap          |ap2         |band|time_epoch|rssi |
+------------------------------------+------------+------------+----+----------+-----+
|ee915a44-b963-11e5-b14c-1258369c38a9|5c5b3550032f|d420b085fb3f|5   |1590076639|-83.0|
|ee915a44-b963-11e5-b14c-1258369c38a9|5c5b350e022b|5c5b350e1455|24  |1590076715|-81.0|
|ee915a44-b963-11e5-b14c-1258369c38a9|5c5b350e022b|d420b085fb3f|5   |1590076715|-73.0|
|ee915a44-b963-11e5-b14c-1258369c38a9|5c5b350e1455|d420b085fb3f|24  |1590076690|-61.0|
|ee915a44-b963-11e5-b14c-1258369c38a9|5c5b3550032f|5c5b350e022b|5   |1590076639|-67.0|
|ee915a44-b963-11e5-b14c-1258369c38a9|5c5b3550032f|5c5b350e1455|24  |1590076639|-71.0|
|ee915a44-b963-11e5-b14c-1258369c38a9|5c5b3550032f|d420b085fc2f|5   |1590076639|-84.0|
|ee915a44-b963-11e5-b14c-1258369c38a9|5c5b3550032f|5c5b350eb1fe|5   |1590076639|-80.0|
|ee915a44-b963-11e5-b14c-1258369c38a9|5c5b3

In [48]:

impact_aps = df_radar_in_site.select("ap_id").distinct().collect()

In [49]:
impacted_ap_id = impact_aps[0]['ap_id']

In [50]:
df_scan_neighbor = df_scan_in_site.filter((col("ap")==impacted_ap_id)&(col('rssi')>-75)&(col("ap2")!=''))\
                                .select('ap2', 'rssi').groupBy("ap2").agg({"rssi": 'max'})

df_scan_neighbor.show()

+------------+---------+
|         ap2|max(rssi)|
+------------+---------+
|d420b085fb3f|    -71.0|
|5c5b350e022b|    -54.0|
|5c5b350eb1fe|    -69.0|
|5c5b350e1455|    -66.0|
+------------+---------+

In [51]:
# df_scan_neighbor.select('ap2', 'rssi').groupBy("ap2").agg({"rssi": 'max'}).show()
df_scan_neighbor = df_scan_neighbor.select(col("ap2").alias("ap"), col("max(rssi)").alias("max_rssi"))
df_scan_neighbor.show()

+------------+--------+
|          ap|max_rssi|
+------------+--------+
|d420b085fb3f|   -71.0|
|5c5b350e022b|   -54.0|
|5c5b350eb1fe|   -69.0|
|5c5b350e1455|   -66.0|
+------------+--------+

In [52]:

# weight_neighbor = udf(lambda r: weight_impacted_neighbor(r, -65))
# df_scan_neighbor_weight = df_scan_neighbor.withColumn("weight", weight_channel(col('max_rssi')))

# df_scan_neighbor_weight.show()


name 'weight_channel' is not defined
Traceback (most recent call last):
NameError: name 'weight_channel' is not defined



# Code optimization (Shirley)

In [68]:
rrm_radar_df.show()

+--------------------+------------+-----------+-------------------+---------------+----------+
|             site_id|       ap_id|pre_channel|radar_channel_count|last_radar_time|      time|
+--------------------+------------+-----------+-------------------+---------------+----------+
|c943d077-1b67-491...|5c5b35500622|        104|                  1|     1589096727|1589094000|
|c943d077-1b67-491...|5c5b35500924|        116|                  1|     1589095299|1589094000|
|c943d077-1b67-491...|5c5b35500686|        104|                  1|     1589095247|1589094000|
|c943d077-1b67-491...|5c5b35500587|         64|                  1|     1589096287|1589094000|
|c943d077-1b67-491...|5c5b35500839|         64|                  1|     1588919882|1588917600|
|c943d077-1b67-491...|5c5b35500686|         64|                  1|     1588920367|1588917600|
|c943d077-1b67-491...|5c5b35500622|        108|                  1|     1588921154|1588917600|
|c943d077-1b67-491...|5c5b35500609|        120|   

In [65]:
from pyspark.sql import functions as F
scan_df = df_scan
rrm_radar_df = df_radar

# env = "staging"

# s3_bucket = "s3://mist-aggregated-stats-{env}/aggregated-stats/".format(env=env)
# date_day = "2020-05-*"
# hr = "*" #now().hour-1

# rrm_radar_path = s3_bucket + "rrm_radar_channel_stat/dt={day}/hr={hr}/*.csv".format(env=env, day=date_day, hr=hr)
# print(rrm_radar_path)


# date_day = datetime.today().strftime('%Y-%m-%d') #"2020-05-*"
# curr_hr = '*'  # latest 
# ap_neighbors_path = s3_bucket + "top_1_time_epoch_by_site_ap_ap2_band/dt={day}/hr={hr}/*.csv".format(env=env, day=date_day, hr=hr)

# print(ap_neighbors_path)


scan_df = spark.read.csv(ap_neighbors_path, header=True)                   
rrm_radar_df = spark.read.csv(rrm_radar_path, header=True)    
    

ap_neighbor_radar = scan_df\
             .filter('ap2 is not Null and rssi >-65')\
             .join(rrm_radar_df, [scan_df.site == rrm_radar_df.site_id, scan_df.ap2 == rrm_radar_df.ap_id], 'inner')\
             .select('ap', 'ap2', 'pre_channel', 'radar_channel_count').groupBy('ap', 'pre_channel')\
             .agg(F.collect_list('ap2').alias('neighbor_aps'), F.collect_list('radar_channel_count').alias('radar_channel_counts'))\
            .withColumnRenamed('pre_channel', 'neighbor_radar_channel')


In [72]:
ap_neighbor_radar.show(1, False)

+------------+----------------------+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+---------------------------------------------------------------+
|ap          |neighbor_radar_channel|neighbor_aps                                                                                                                                                                                                                                                                                          |radar_channel_counts                                           |
+------------+----------------------+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [75]:
ap_neighbor_radar.select('neighbor_aps', 'radar_channel_counts').show(1, False)

+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+---------------------------------------------------------------+
|neighbor_aps                                                                                                                                                                                                                                                                                          |radar_channel_counts                                           |
+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [69]:
final_df = rrm_radar_df.join(ap_neighbor_radar, [rrm_radar_df.ap_id == ap_neighbor_radar.ap, rrm_radar_df.pre_channel == ap_neighbor_radar.neighbor_radar_channel], 'outer')



In [71]:
final_df.show(3)

+--------------------+------------+-----------+-------------------+---------------+----------+------------+----------------------+--------------------+--------------------+
|             site_id|       ap_id|pre_channel|radar_channel_count|last_radar_time|      time|          ap|neighbor_radar_channel|        neighbor_aps|radar_channel_counts|
+--------------------+------------+-----------+-------------------+---------------+----------+------------+----------------------+--------------------+--------------------+
|35670395-458f-4ec...|5c5b350ec479|        100|                  1|     1588968708|1588968000|        null|                  null|                null|                null|
|                null|        null|       null|               null|           null|      null|5c5b353f574d|                   140|[5c5b350ecfeb, 5c...|[1, 1, 1, 1, 1, 1...|
|c943d077-1b67-491...|5c5b355008b6|         64|                  1|     1589106042|1589104800|        null|                  null|     

In [ ]:
final_df.sele